In [1]:
!pip install pyspark

In [2]:
!git clone https://github.com/wandersondsm/teste_engenheiro

Cloning into 'teste_engenheiro'...
remote: Enumerating objects: 1189, done.
remote: Total 1189 (delta 0), reused 0 (delta 0), pack-reused 1189 (from 2)
Receiving objects: 100% (1189/1189), 49.85 MiB | 9.71 MiB/s, done.
Resolving deltas: 100% (1148/1148), done.
Updating files: 100% (1182/1182), done.


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [4]:
spark = SparkSession.builder.appName('Create Patient').getOrCreate()

In [5]:
spark

## Leitura dos dados e visualização geral

In [6]:
df = spark.read.option('multiline','true').json('/content/teste_engenheiro/data/*.json')

In [8]:
df.show(5)

+--------------------+------------+-----------+
|               entry|resourceType|       type|
+--------------------+------------+-----------+
|[{urn:uuid:071812...|      Bundle|transaction|
|[{urn:uuid:df121e...|      Bundle|transaction|
|[{urn:uuid:f156b8...|      Bundle|transaction|
|[{urn:uuid:b0f49c...|      Bundle|transaction|
|[{urn:uuid:37ff59...|      Bundle|transaction|
+--------------------+------------+-----------+
only showing top 5 rows



In [9]:
df.count()

1180

In [10]:
df_patient = df.select(F.explode('entry'))

In [11]:
df_patient.count()

527113

In [12]:
df_patient.show(5,False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [13]:
df_save = df_patient.withColumn('RESOURCE_TYPE', F.col('col.resource.resourceType'))\
                    .where("RESOURCE_TYPE like 'Patient'")\
                    .withColumn('ID', F.col('col.resource.id'))\
                    .withColumn('GENDER', F.col('col.resource.gender'))\
                    .withColumn('BIRTHDATE', F.col('col.resource.birthdate').cast('timestamp'))\
                    .withColumn('MARITAL_STATUS', F.col('col.resource.maritalStatus.text'))\
                    .withColumn('MULTIPLE_BIRTH_BOOLEAN', F.col('col.resource.multipleBirthBoolean').cast('boolean'))\
                    .drop('col','RESOURCE_TYPE')

In [14]:
df_save.printSchema()

root
 |-- ID: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- BIRTHDATE: timestamp (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- MULTIPLE_BIRTH_BOOLEAN: boolean (nullable = true)



In [15]:
df_save.show(10,False)

+------------------------------------+------+-------------------+--------------+----------------------+
|ID                                  |GENDER|BIRTHDATE          |MARITAL_STATUS|MULTIPLE_BIRTH_BOOLEAN|
+------------------------------------+------+-------------------+--------------+----------------------+
|0718123b-5034-4965-a145-3d8d71b11389|male  |1917-08-06 00:00:00|M             |false                 |
|df121e33-f3dc-4d02-a523-3bbd89c8fa5b|male  |1924-06-30 00:00:00|M             |false                 |
|f156b85a-67c6-4238-96b7-3e869f0462d2|male  |1912-10-17 00:00:00|M             |false                 |
|b0f49c80-b59b-4df6-8292-40ce8b8f8612|female|1927-06-20 00:00:00|M             |false                 |
|37ff5999-5a5a-472d-9b2c-408693f101fa|male  |1951-05-24 00:00:00|M             |false                 |
|cbaf7927-acab-449a-b3bb-01f02a8d4966|male  |1911-04-08 00:00:00|M             |false                 |
|32fde49e-97b8-483a-a818-e67090edbf53|male  |1958-03-22 00:00:00

In [16]:
df_save.count()

1180

In [17]:
df_save.coalesce(1).write.option('header',True).option('delimiter',';').csv('patient')